# Application of SVD: Latent Semantic Analysis

Strategy: given a corpus of articles, we want to create a term-document-type of matrix, for which we can do SVD analysis

In [1]:
import pickle
import os
import time

import numpy as np
import scipy.sparse.csr as csr
import scipy.sparse as sparse
from sklearn.base import clone
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.neighbors import KNeighborsClassifier

import matplotlib.pyplot as plt
%matplotlib inline

# I. Representing a corpus of documents as a matrix

## Bag of Words matrix

Following the <a href="http://scikit-learn.org/stable/modules/feature_extraction.html">Sklearn Feature-extraction documentation page</a>

- we start with a given **corpus** of $D$ documents.
- we preprocess each document and convert it into a list of terms (features)
    - by lowercasing first
    - accepting only word patterns (defined via regex)
- then we form the $CV$ Count-Vectorizer term-frequency matrix defined as:

$$
\text{tf}(t, d)\equiv{CF}_{d,t} = \text{# times term }t\text{ occurs in document }d
$$




In [34]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    token_pattern='(?u)\\b\\w\\w+\\b', #default
#     token_pattern='(?u)\\b[a-zA-Z]\\w+\\b',    
)
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [37]:
corpus = [
    'This is the the first first document abra.',
    'This is the second second document cadabra.',
    'And the third one 3.',
    'Is this the first document 4?',
]
X_corpus_docterm = vectorizer.fit_transform(corpus)
# note the effect of modifying the token_pattern above....
features = vectorizer.get_feature_names() 
features

['abra',
 'and',
 'cadabra',
 'document',
 'first',
 'is',
 'one',
 'second',
 'the',
 'third',
 'this']

In [38]:
# This is our document-term matrix CV:
CV = X_corpus_docterm.toarray()
CV

array([[1, 0, 0, 1, 2, 1, 0, 0, 2, 0, 1],
       [0, 0, 1, 1, 0, 1, 0, 2, 1, 0, 1],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1]], dtype=int64)

In [41]:
def doc2vec(doc, orig_vectorizer, debug=False):
    # We can represent any new document in terms of our previous model
    doc_vectorizer = clone(orig_vectorizer)
    doc_corpus = [doc]
    X_docterm = doc_vectorizer.fit_transform(doc_corpus)
    doc_features = doc_vectorizer.get_feature_names()
    doc_counts = X_docterm.toarray()[0]
    if debug:
        print(doc_features, doc_counts)
    nonzero_idx = [
        orig_vectorizer.vocabulary_.get(feature) for feature in doc_features]
    vec = sparse.coo_matrix(
        (doc_counts, (np.zeros_like(doc_counts), nonzero_idx)),
        shape = (1, CV.shape[1]))
    return vec, doc_features, doc_counts

v,_,_ = doc2vec('The the first', vectorizer, debug=True)
v, v.toarray()

['first', 'the'] [1 2]


(<1x11 sparse matrix of type '<class 'numpy.int64'>'
 	with 2 stored elements in COOrdinate format>,
 array([[0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0]], dtype=int64))

In [42]:
# To recap
corpus, np.array(features), X_corpus_docterm.toarray()

(['This is the the first first document abra.',
  'This is the second second document cadabra.',
  'And the third one 3.',
  'Is this the first document 4?'],
 array(['abra', 'and', 'cadabra', 'document', 'first', 'is', 'one',
        'second', 'the', 'third', 'this'],
       dtype='<U8'),
 array([[1, 0, 0, 1, 2, 1, 0, 0, 2, 0, 1],
        [0, 0, 1, 1, 0, 1, 0, 2, 1, 0, 1],
        [0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0],
        [0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1]], dtype=int64))

### Problems with Term-Document matrix:

- The document vectors are not normalized 
    - can't really compare documents
- The document vectors contain many common english words containing no information
    - ideally we want to remove those, e.g. 'the', 'is', etc


### Solution: TF-IDF vectorizer (see <a href=http://scikit-learn.org/stable/modules/feature_extraction.html> The TF-idf section in the Scikit-Learn feature extraction manual</a>)

Instead, let's consider the following matrix

$$
\begin{align}
\text{tf-idf}(t,d) &\equiv{\text{tf}}(t,d)\times\text{idf}(t)\\
\text{idf}(t)&\equiv\log\frac{1+n_d}{1+\text{df}(d,t)} + 1
\end{align}
$$

where 

- $\text{df}(d,t)$ is the number of documents containing feature $t$
- the rows of the tf-idf matrix are normalized to have unit norm (either $L_1$ or $L_2$)
    - this way we can compare documents by the norm of their doc2vec overlaps
    
Let's see this in practice

In [43]:
vectorizer = TfidfVectorizer(
    stop_words='english',
    norm='l2',
    use_idf=True)

X_corpus_tfidf=vectorizer.fit_transform(corpus)

# Note that only 2 vectors survive the transformation
vectorizer.get_feature_names() 

['abra', 'cadabra', 'document', 'second']

In [44]:
X_corpus_tfidf.toarray()

array([[ 0.84292635,  0.        ,  0.53802897,  0.        ],
       [ 0.        ,  0.43003652,  0.27448674,  0.86007303],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.        ,  0.        ]])

In [45]:
# Note that the no-zero rows are all normalized to 1
X_corpus_tfidf.toarray() ** 2

array([[ 0.71052483,  0.        ,  0.28947517,  0.        ],
       [ 0.        ,  0.18493141,  0.07534297,  0.73972562],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.        ,  0.        ]])

In [46]:
v,_,_ = doc2vec('The the first abra', vectorizer, debug=True)
v, v.toarray()

['abra'] [ 1.]


(<1x11 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in COOrdinate format>,
 array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]))

# II. Latent Semantic Analysis

## Or Truncated SVD on the TF-IDF matrix

**The Following code is based on <a href="http://scikit-learn.org/stable/auto_examples/applications/plot_out_of_core_classification.html">Scikit-Learn's Reuters Dataset TF-IDF + K-NN classification example</a> along with <a href="http://mccormickml.com/2016/03/25/lsa-for-text-classification-tutorial/">Chris McCormic's LSA tutorial</a> and his <a href="https://github.com/chrisjmccormick/LSA_Classification">github page</a>. The original Reuter's 21578 dataset is part of the <a href="http://archive.ics.uci.edu/ml/machine-learning-databases">UCI-ML</a> repository and can be found <a href="http://archive.ics.uci.edu/ml/machine-learning-databases/reuters21578-mld/reuters21578.tar.gz">here</a>. However, in this demo we are using the already <a href="https://github.com/chrisjmccormick/LSA_Classification/tree/master/data">pre-processed</a> version in Chris McCormic's github page**

### Let's look at some real data - the Reuters Articles Corpus


In [47]:
fname = "raw_text_dataset.pickle"
filepath = os.getcwd() + '/' + fname
raw_text_dataset = pickle.load(open(filepath, "rb"))
X_train_raw = raw_text_dataset[0]
y_train_labels = raw_text_dataset[1] 
X_test_raw = raw_text_dataset[2]
y_test_labels = raw_text_dataset[3]

print('Number of train docs:', len(X_train_raw), 
      'Number of test docs:', len(X_test_raw))
print('\ntrain labels:', y_train_labels[:4])
print('\nThis is how a sample train article looks like:\n\n', 
      X_train_raw[0][:500])

Number of train docs: 4743 Number of test docs: 4858

train labels: [['cocoa', 'el-salvador', 'usa', 'uruguay'], ['usa'], ['usa'], ['usa', 'brazil']]

This is how a sample train article looks like:

 BAHIA COCOA REVIEW

Showers continued throughout the week in the Bahia cocoa zone, alleviating the drought since early January and improving prospects for the coming temporao, although normal humidity levels have not been restored, Comissaria Smith said in its weekly review. The dry period means the temporao will be late this year. Arrivals for the week ended February 22 were 155,221 bags of 60 kilos making a cumulative total for the season of 5.93 mln against 5.81 at the same stage last year. A


### TF-IDF vectorizer step:
The TfidfVectorizer below does the following:
- TF Step
    - Strips out “stop words”, e.g. frequently occuring english words
    - Filters out terms that occur in more than half of the docs
    (max_df=0.5)
    - Filters out terms that occur in only one document (min_df=2).
    - Selects the 10,000 most frequently occuring words in the corpus.
    - Normalizes the vector to account for the effect of document
    length on the tf-idf values. Here we use l1 norm which normalized
    by the document length
- IDF Step
    - Nomalize each 
    


In [48]:
vectorizer = TfidfVectorizer(
    max_df=0.5, # ignore terms which occur in more than half of the documents
    max_features=10000,
    min_df=2, # ignore terms which occur in less than 2 documents
    stop_words='english',
    norm='l2',
    use_idf=True, 
    analyzer='word',
    token_pattern='(?u)\\b\\w\\w+\\b'
#     token_pattern = '(?u)\\b[a-zA-Z]\\w+\\b'
    )

# note how changing the token_pattern changes
X_train_tfidf = vectorizer.fit_transform(X_train_raw)
print(X_train_tfidf.shape)
print('first 10 features:', vectorizer.get_feature_names()[:10])
print('last 10 features:', vectorizer.get_feature_names()[-10:])

(4743, 10000)
first 10 features: ['00', '000', '0000', '001', '002', '003', '004', '005', '006', '008']
last 10 features: ['zimbabwe', 'zinc', 'zoete', 'zone', 'zones', 'zorinsky', 'zortman', 'zuckerman', 'zurich', 'zy']


In [49]:
# let's look at documents that contain the word 'cocoa'
doc_idx = X_train_tfidf[
    :, vectorizer.vocabulary_.get('bullish')].nonzero()[0].tolist()
print(len(doc_idx), ' documents found')
i = 1
print('document ', i, ':')
X_train_raw[doc_idx[i]]

19  documents found
document  1 :


"TALKING POINT/AUTO SUPPLIER STOCKS\n\nThe 1987 outlook for U.S. auto sales is clouded by a decidedly mixed sales forecast, and analysts who follow the industry say it is also true for companies that sell parts and equipment to the major car and truck manufacturers. But while there are only four major U.S.-based automakers whose shares are traded on stock exchanges, there are thousands of big and small suppliers who sell a flood of original and replacement parts. Analysts who follow the parts industry say there are many opportunities for investors brought on by the auto industry's intensified competition and the large volume of production in North America planned by Japanese automakers. But assessing the supplier arena is far more complicated than for investors considering the stocks of the Detroit Big Three - General Motors Corp GM>, Ford Motor Co F> and Chrysler Corp C>. Despite widespread predictions that U.S. vehicle sales will decline about 10 pct from the record 1987 levels, Wall

In [52]:
print("\nPerforming dimensionality reduction using LSA")
t0 = time.time()

# Project the tfidf vectors onto the first N principal components.
# Though this is significantly fewer features than the original tfidf vector,
# they are stronger features, and the accuracy is higher.
svd = TruncatedSVD(
    n_components=200,
    random_state=42,
    algorithm='arpack'
)

lsa = make_pipeline(
    svd, 
#     Normalizer(copy=False) # try commenting this out. Do you get a better result?
)

# Run SVD on the training data, then project the training data.
X_train_lsa = lsa.fit_transform(X_train_tfidf)

print("  done in %.3fsec" % (time.time() - t0))


Performing dimensionality reduction using LSA
  done in 4.397sec


In [57]:
explained_variance = svd.explained_variance_ratio_.sum()
print("  Explained variance of the SVD step: {}%".format(int(explained_variance * 100)))


# Now apply the transformations to the test data as well.
X_test_tfidf = vectorizer.transform(X_test_raw)
X_test_lsa = lsa.transform(X_test_tfidf)



  Explained variance of the SVD step: 36%


### Let's improve a K-nn classifier using LSA

In [54]:
# The Reuters dataset consists of ~100 categories. However, we are going to
# simplify this to a binary classification problem. The 'positive class' will
# be the articles related to "acquisitions" (or "acq" in the dataset). All
# other articles will be negative.
y_train = ["acq" in y for y in y_train_labels]
y_test = ["acq" in y for y in y_test_labels]

In [59]:
print("\nClassifying tfidf vectors...")

# Time this step.
t0 = time.time()

# Build a k-NN classifier. Use k = 5 (majority wins), the cosine distance, 
# and brute-force calculation of distances.
knn_tfidf = KNeighborsClassifier(n_neighbors=5, algorithm='brute', metric='cosine')
knn_tfidf.fit(X_train_tfidf, y_train)

# Classify the test vectors.
p = knn_tfidf.predict(X_test_tfidf)

# Measure accuracy
numRight = 0;
for i in range(0,len(p)):
    if p[i] == y_test[i]:
        numRight += 1

print("  (%d / %d) correct - %.2f%%" % (numRight, len(y_test), float(numRight) / float(len(y_test)) * 100.0))

# Calculate the elapsed time (in seconds)
elapsed = (time.time() - t0)
print("  done in %.3fsec" % elapsed)


Classifying tfidf vectors...
  (4471 / 4858) correct - 92.03%
  done in 1.863sec


In [60]:
print("\nClassifying LSA vectors...")

# Time this step.
t0 = time.time()

# Build a k-NN classifier. Use k = 5 (majority wins), the cosine distance, 
# and brute-force calculation of distances.
knn_lsa = KNeighborsClassifier(n_neighbors=5, algorithm='brute', metric='cosine')
knn_lsa.fit(X_train_lsa, y_train)

# Classify the test vectors.
p = knn_lsa.predict(X_test_lsa)

# Measure accuracy
numRight = 0;
for i in range(0,len(p)):
    if p[i] == y_test[i]:
        numRight += 1

print("  (%d / %d) correct - %.2f%%" % (numRight, len(y_test), float(numRight) / float(len(y_test)) * 100.0))

# Calculate the elapsed time (in seconds)
elapsed = (time.time() - t0)    
print("    done in %.3fsec" % elapsed)



Classifying LSA vectors...
  (4561 / 4858) correct - 93.89%
    done in 1.086sec


# Homework Probelm 1:
- Download the Reuters data from Chris McCormick's github repo https://github.com/chrisjmccormick/LSA_Classification
    - Make sure the code in this notebook runs against the raw_text_dataset.pickle in the github repo
    - assume that the dataset PATH is in the same directory as where the notebook is so that our grader Yifei can run your notebook with his own copy of the dataset in the same folder
    - also feel free to modify this current notebook so as to complete the rest of the problem below:
    
- Create a doc2vec(doc, tfidf_vectorizer) function corresponding to a TFIDF vec
    - INPUTS: doc, tfidf_vectorizer
        - doc - any string
        - tfidf_vectorizer - a TfidfVectorizer instance
    - OUTPUTS: vec, doc_features, doc_counts
        - vec - a vector with $L_2$ norm of $1$
        - doc_features - the features after tokenization and pre-processing
        - doc_counts - the counts of each feature in this document
    - note that you should not normalize by the inverse document frequency as there is just a single document
- For each of the following doc strings, calculate their corresponding vectors
    - doc1: "The cocoa cadabra"
    - doc2: "AAPL SE"
    - doc3: "bullish stocks"
- Create a **recommend(vec, X_model, X_corpus)** function:
    - which projects any document vector onto X_model
        - here X_model = {X_train_tfidf, and X_train_lsa}
    - and returns doc_vec, idx_top10, sim_top10, X_top10 as follows
        - doc_vec - the (sparse) vector of similarity scores of vec and members of X_model. 
        This vector should be size (Dx1)
        - idx_top10: the indices of the top-10 similarity scores
        - sim_top10: the top-10 similarity scores
        - X_top10: the top-10 corpus articles most similar to the input model
    - what does your recommend() function ouput for the doc vectors in the previous exersise?
        - Do you see an improvement of the LSA similarity recommendation relative to the TF-IDF similarity recommendation?
